In [8]:
import napari 
import pandas as pd
import numpy as np
import math

from skimage.io import imread
from skimage.measure import regionprops_table
from skimage.measure import marching_cubes, mesh_surface_area 

from scipy.ndimage import binary_fill_holes, binary_closing, grey_dilation, binary_dilation


import matplotlib.pyplot as plt

# for processing the meshes 
import open3d as o3d
import pymeshfix
from pymeshfix._meshfix import PyTMesh

import vedo 


from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import os 

In [14]:
metrics_3d = [
#     'area',
# #  'bbox',
# #  'bbox_area',
# # 'convex_area', # cause error
# #  'convex_image',
#  'coords',
# 'feret_diameter_max',
#  'filled_area',
#  'filled_image',
#  'image',
#   'inertia_tensor',
#   'inertia_tensor_eigvals',
# 'label',
 'major_axis_length',
 'minor_axis_length',
 'extent', 
#   'moments',
#  'moments_central',
 # 'slice',
 'solidity'
] 

def get_measurements(masks, metrics, surf): 
    # measure our data 
    shapes_2 = regionprops_table(masks, properties = metrics)
    df = pd.DataFrame(shapes_2)

    df['surface area'] = surf

    df['cuboidness'] = df['area'] / masks.shape[0] * masks.shape[1] * masks.shape[2]
    df['sphericity'] = (math.pi**(1/3) * (6 * df['area'])**(2/3)) / df['surface area'] 
    df['flatness'] = df['major_axis_length'] / df['minor_axis_length'] 
    
    return(df)

def get_surface_area(tb_mask): 
    surface_areas = []

    for i in np.unique(tb_mask.flatten())[np.unique(tb_mask.flatten())>0]: 
        #print(i)
        mask = tb_mask.copy()
        mask[mask != i] = 0

        verts, faces, _, __ = marching_cubes(mask)

        surface_areas.append(mesh_surface_area(verts, faces))
    return(surface_areas) 

def calculate_metrics(df): 
    # calculate ap, dv, ml distances 

    # none of this is sensible if the embryo isn't aligned right which it aint 

    # df['ap'] = (df['bbox-4'] - df['bbox-1']) 
    # df['dv'] = (df['bbox-3'] - df['bbox-0'])
    # df['ml'] = (df['bbox-5'] - df['bbox-2'])
    # # calculate ratios 
    # df['AP:DV'] = df['ap'] / df['dv']
    # df['AP:ML'] = df['ap'] / df['ml']
    # df['DV:ML'] = df['dv'] / df['ml'] 
    
    # # transverse volume ratios - may need to remove this 
    # df['transverse'] = (df['dv'] + df['ml']) / 2 #transverse cell diameter - why is this useful? 
    # df['AP_anisotropy'] = df['ap'] / df['transverse'] 
    
    # df['transverse_spread'] = df['dv'] * df['ml'] 
    # df['transverse_xsection'] = df['area'] / df['ap'] 
    
    # shape metrics 
    df['cuboidness'] = df['area'] / (df['ap'] * df['dv'] * df['ml']) 
    df['sphericity'] = (math.pi**(1/3) * (6 * df['area'])**(2/3)) / df['surface area'] 
    df['flatness'] = df['major_axis_length'] / df['minor_axis_length'] 
    
    return(df)

def do_all_metrics(mask, metrics, surf):
    df = get_measurements(mask, metrics)
    # surf = get_surface_area(mask)
    df['surface area'] = surf
    df = calculate_metrics(df)
    return(df)


In [16]:
cell

'cell_10001'

In [15]:
data = []

cell_list = os.listdir('../output/cell_segs_limeseg/Img_1_segmentation2.0/')

print('started')

for cell in cell_list[0:3]: 
    path = '../output/cell_segs_limeseg/Img_1_segmentation2.0/' + cell + '/T_1.ply_.npy'
    if os.path.exists(path): 
        img = np.load(path)
        d = get_measurements(img, metrics_3d, surf = 1) # oops forgot to save this 
        data.append(d)

    # helpful to view the pointcloud directly 
    # if the segmentation isn't working
    # viewer.add_points(np.asarray(pointcloud.points))
    # pointcloud = o3d.io.read_point_cloud(path)
    # pointcloud.get_min_bound().astype(int)

started


KeyError: 'area'